In [1]:
print("Hola")

Hola


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from ipywidgets import interact, widgets

In [3]:
data = pd.read_csv("Base De Datos Pizza (Respuestas).csv")

In [4]:
columnas_sabores = ['Carne Molida', 'Chorizo', 'Jamón', 'Pepperoni', 'Pollo', 'Salami', 'Tocineta', 'Aceituna', 'Cebolla', 'Champiñón', 'Ciruela']

# Combinar los sabores de las columnas en una sola columna
sabores = data[columnas_sabores].apply(lambda x: ' '.join(x.astype(str)), axis=1)

# Crear un vectorizador TF-IDF con sublinear_tf=True para obtener valores continuos
vectorizer = TfidfVectorizer(sublinear_tf=True, token_pattern=r'\b[1-9][0-9]?\b')

# Transformar los sabores en una matriz TF-IDF
tfidf_matrix = vectorizer.fit_transform(sabores)

# Calcular la similitud de coseno entre los sabores de pizza
similarity_matrix = cosine_similarity(tfidf_matrix)

# Obtener los nombres de las personas
nombres = data['Nombre']

k = 3  # Ajusta el número de vecinos que deseas obtener

# Inicializar el modelo de búsqueda de vecinos más cercanos
model = NearestNeighbors(n_neighbors=k, metric='precomputed')

# Ajustar el modelo utilizando la matriz de similitud
model.fit(similarity_matrix)

# Encontrar los k vecinos más cercanos para cada punto
distances, indices = model.kneighbors(similarity_matrix)

def generate_graph(person_name):
    # Find the index of the person in the 'Nombre' column
    person_index = data[data['Nombre'] == person_name].index[0]

    # Get the neighbors for the person of interest
    person_neighbors = [(nombres[idx], distances[person_index][j]) for j, idx in enumerate(indices[person_index])]
    person_neighbors.sort(key=lambda x: x[1])  # Sort by ascending similarity

    print(f"Neighbors of {person_name}:")
    for neighbor in person_neighbors:
        print(f"Neighbor: {neighbor[0]}, Similarity: {neighbor[1]}")

    # Create a bar plot to visualize the similarities of the neighbors
    plt.figure()
    plt.bar(range(len(person_neighbors)), [d[1] for d in person_neighbors])
    plt.xticks(range(len(person_neighbors)), [d[0] for d in person_neighbors], rotation=45)
    plt.xlabel('Neighbor')
    plt.ylabel('Similarity')
    plt.title(f"Neighbors of {person_name}")
    plt.show()

def on_button_clicked(b):
    person_name = dropdown.value
    generate_graph(person_name)

dropdown = widgets.Dropdown(options=nombres, description='Select a person:')
button = widgets.Button(description='Enter')

button.on_click(on_button_clicked)

widgets.VBox([dropdown, button])